In [1]:
# The only 3 lines you need to install and use AMPL with any solver on Colab
!pip install -q amplpy
from amplpy import tools
ampl = tools.ampl_notebook(
    modules=["highs", "gurobi"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="your-license-uuid") # use your AMPL Community Edition License UUID to gain access to commercial solver trials

Please provide a valid license UUID. You can use a free https://ampl.com/ce license.


In [2]:
%%ampl_eval
reset;
set ITEMS;

param weight{ITEMS};
param value{ITEMS};
param max_weight;

var x{ITEMS} binary;

maximize total_value: sum{i in ITEMS} value[i] * x[i];

subject to weight_constraint: sum{i in ITEMS} weight[i] * x[i] <= max_weight;

In [3]:
from random import randint
items = list(range(1, 10+1))
ampl.set['ITEMS'] = items
ampl.param['weight'] = {i: randint(10, 20) for i in items}
ampl.param['value'] = {i: randint(100, 200) for i in items}
ampl.param['max_weight'] = 100
ampl.option["solver"] = "highs"
ampl.solve()
ampl.get_data("x").to_pandas()

HiGHS 1.5.1: HiGHS 1.5.1: optimal solution; objective 1157
1 simplex iterations
1 branching nodes


,x
1.0,0.0
2.0,1.0
3.0,1.0
4.0,1.0
5.0,1.0
6.0,0.0
7.0,1.0
8.0,1.0
9.0,1.0
10.0,0.0


[AMPL Website](https://ampl.com) | [AMPL Colab](https://colab.ampl.com) | [Community Edition]( https://ampl.com/ce/) | [Twitter](https://twitter.com/AMPLopt) | [LinkedIn](https://www.linkedin.com/company/ampl)

[![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/template/minimal.ipynb)](https://colab.ampl.com)